In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
from prophet import Prophet
from prophet.diagnostics import cross_validation
import itertools
from prophet.diagnostics import performance_metrics

bus_data = pd.read_csv("municipality_bus_utilization.csv", parse_dates= ['timestamp'])

In [2]:
bus_data.isna().sum()

In [3]:
bus_data1 = bus_data[(bus_data['municipality_id'] == 0)]

In [5]:
bus_data1

In [7]:
plt.plot(bus_data1.timestamp, bus_data1.usage)
plt.xticks(rotation=30, ha='right')
 
plt.title('Usage of Municipality 0')
 
plt.xlabel('Date')
plt.ylabel('Usage')
plt.figure(figsize=(650, 2))
plt.tight_layout()
plt.show()

In [8]:
tot_readings = 1307 # total number of rows in your df
# find max per hour:
records_per_dt = 2
max_values_per_dt = pd.DataFrame({}) # final data frame where to store maxs and times
for h in np.arange(0, tot_readings, records_per_dt):
    df_sliced = bus_data1[h:h+records_per_dt] # takes only minutes_per_dt rows starting from h
    tmp_max = pd.DataFrame(df_sliced.max()).transpose() # here you have a row with the time and the max values for the current hour analysed in the loop
    max_values_per_dt = max_values_per_dt.append(tmp_max) # append the maxs and the times into your final dataframe

In [9]:
max_values_per_dt

In [10]:
plt.plot(max_values_per_dt.timestamp, max_values_per_dt.usage)
plt.xticks(rotation=30, ha='right')
 
plt.title('Usage of Municipality 0')
 
plt.xlabel('Date')
plt.ylabel('Usage')
plt.figure(figsize=(6, 1))
plt.tight_layout()
plt.show()

In [12]:
max_values_per_dt['timestamp'] =  pd.to_datetime(max_values_per_dt['timestamp'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

In [13]:
max_values_per_dt.dtypes

In [14]:
max_values_per_dt.isna().sum()

In [16]:
max_values_per_dt.isna().sum()

In [17]:
max_values_per_dt['year']=max_values_per_dt.timestamp.dt.year
max_values_per_dt['month']=max_values_per_dt.timestamp.dt.month 
max_values_per_dt['day']=max_values_per_dt.timestamp.dt.day    
max_values_per_dt['Hour']=max_values_per_dt.timestamp.dt.hour

In [18]:
def applyer(row):
    if row.dayofweek == 5 or row.dayofweek == 6:
        return 1
    else:
        return 0
temp2 = max_values_per_dt['timestamp'].apply(applyer) 
max_values_per_dt['weekend']=temp2

In [19]:
max_values_per_dt.index = max_values_per_dt['timestamp'] 
df=max_values_per_dt.drop('municipality_id',1) 
ts = df['usage'] 
plt.figure(figsize=(16,8)) 
plt.plot(ts, label='Passenger Count') 
plt.title('Time Series') 
plt.xlabel("Time(year-month)") 
plt.ylabel("Passenger count") 
plt.legend(loc='best')

In [20]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index.
    """
    df = df.copy()
    df['timestamp'] = df.index
    print(max_values_per_dt.dtypes)
    df['hour'] = df['timestamp'].dt.hour
    df['dayofweek'] = df['timestamp'].dt.dayofweek
    df['quarter'] = df['timestamp'].dt.quarter
    df['month'] = df['timestamp'].dt.month
    df['year'] = df['timestamp'].dt.year
    df['dayofyear'] = df['timestamp'].dt.dayofyear
    df['dayofmonth'] = df['timestamp'].dt.day
    df['weekofyear'] = df['timestamp'].dt.weekofyear
    
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    if label:
        y = df[label]
        return X, y
    return X

X, y = create_features(max_values_per_dt, label='usage')

features_and_target = pd.concat([X, y], axis=1)

In [21]:
max_values_per_dt

In [22]:
sns.pairplot(features_and_target.dropna(),
             hue='hour',
             x_vars=['hour','dayofweek',
                     'year','weekofyear'],
             y_vars='usage',
             height=5,
             plot_kws={'alpha':0.15, 'linewidth':0}
            )
plt.suptitle('Bus Usage by Hour, Day of Week, Year and Week of Year')
plt.show()

In [23]:
split_date = '2017-08-01 00:00:00'
municipality_train = max_values_per_dt.loc[max_values_per_dt.index <= split_date].copy()
municipality_test = max_values_per_dt.loc[max_values_per_dt.index > split_date].copy()

In [24]:
municipality_train = municipality_train.drop(['timestamp', 'total_capacity', 'year', 'month', 'day', 'Hour', 'weekend', 'municipality_id'], axis=1)
municipality_test = municipality_test.drop(['timestamp', 'total_capacity', 'year', 'month', 'day', 'Hour', 'weekend', 'municipality_id'], axis=1)

In [25]:
# Plotting train and test so we can see where we have split
municipality_test \
    .rename(columns={'usage': 'TEST SET'}) \
    .join(municipality_train.rename(columns={'usage': 'TRAINING SET'}),
          how='outer') \
    .plot(figsize=(15,5), title='Municipality 1 Bus Usage', style='.')
plt.show()

In [26]:
# Format data for prophet model using ds and y since Prophet expects the dataset to be named in a specific way
municipality_train.reset_index() \
    .rename(columns={'timestamp':'ds',
                     'usage':'y'}).head()

In [27]:
# Setting up and training model and fitting
model = Prophet()
model.fit(municipality_train.reset_index() \
              .rename(columns={'timestamp':'ds',
                               'usage':'y'}))

# Predicting on training set with model
municipality_test_fcst = model.predict(df=municipality_test.reset_index() \
                                   .rename(columns={'timestamp':'ds'}))

In [28]:
# Plotting the forecast
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
fig = model.plot(municipality_test_fcst,
                 ax=ax)
plt.show()

In [29]:
# Plotting the components of the model
fig = model.plot_components(municipality_test_fcst)

In [30]:
# Plot the forecast with the actuals
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
ax.scatter(municipality_test.index, municipality_test['usage'], color='r')
fig = model.plot(municipality_test_fcst, ax=ax)

In [31]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
ax.scatter(municipality_test.index, municipality_test['usage'], color='r')
fig = model.plot(municipality_test_fcst, ax=ax)
ax.set_xbound(lower='2017-08-01 00:00:00', upper='2017-08-19 16:30:35')
ax.set_ylim(0, 1500)
plot = plt.suptitle('Forecast vs Actuals')

In [32]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print("MAPE of the Forecast(%): ", mean_absolute_percentage_error(y_true=municipality_test['usage'],
                   y_pred=municipality_test_fcst['yhat']))

In [33]:
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5, 1.0],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    'holidays_prior_scale': [0.01, 0.1, 1.0, 10.0],
    'seasonality_mode': ['additive', 'multiplicative']
}
mapes = []

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

for params in all_params:
    m = Prophet(**params).fit(municipality_train.reset_index() \
              .rename(columns={'timestamp':'ds',
                               'usage':'y'}))  # Fit model with given params
    df_cv = cross_validation(m, horizon='1 hour', parallel="processes")
    df_p = performance_metrics(df_cv, rolling_window=1)
    mapes.append(df_p['mape'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['mape'] = mapes
print(tuning_results)

In [34]:
minvalue = tuning_results['mape'].min()
  
minvalue